Test2　　全結合層(linear,relu)のテスト

In [1]:
#　colaboraory用: Google drive をマウントする
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# colaboratory用: フォルダを移る
%cd "drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


フォルダは自分の指定のものに変更して下さい。

In [3]:
%cd "ctorch210929/MiniCTorch_Prototype/notebook"

/content/drive/My Drive/Colab Notebooks/ctorch210929/MiniCTorch_Prototype/notebook


ライブラリのimport

In [4]:
! pip install lark-parser

     |████████████████████████████████| 103 kB 9.2 MB/s 


In [5]:
import sys
sys.path.append("../")

import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import minictorch.generator as GN
import minictorch.converter as CV

ニューラルネットの定義

In [6]:
class Net( nn.Module ):
   def __init__(self):
      super(Net, self).__init__()
      self.fc1 = nn.Linear(2, 32)
      self.fc2 = nn.Linear(32, 32)
      self.fc3 = nn.Linear(32, 3)

   def forward(self, x):
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return x
      #return F.log_softmax(x, dim = 1)

計算グラフをJSONファイルに出力する

In [7]:
def generate_json( json_path, input_x ): 
   
    model = Net()
    print(input_x)

    model.eval()
    with torch.no_grad():
        print("[JSON]", json_path )
        GN.generate_minictorch_file( model, input_to_model, json_path )

    return model

In [8]:
torch.manual_seed( 1 )

input_to_model = torch.randn((1,2),requires_grad=True)

project = "test2"
json_path = './network/' + project +'.json'

model = generate_json( json_path, input_to_model )

tensor([[0.6614, 0.2669]], requires_grad=True)
[JSON] ./network/test2.json
skip: Net/Linear[fc1]/weight/fc1
skip: Net/Linear[fc1]/weight/fc1
skip: Net/Linear[fc2]/weight/fc2
skip: Net/Linear[fc2]/weight/fc2
skip: Net/Linear[fc3]/weight/fc3
skip: Net/Linear[fc3]/weight/fc3


JSONファイルをC++コードに変換する

In [11]:
folder = "./test"
CV.convert_cpp_code( project, folder, model, input_to_model, json_path )

[JSON] ./network/test2.json
{'name': 'Net/Linear[fc1]/weight/weight.7', 'op': 'prim::GetAttr', 'in': [], 'output_id': 0, 'shape': [], 'out': [3], 'sorted_id': 1}
{'name': 'Net/Linear[fc1]/bias/bias.7', 'op': 'prim::GetAttr', 'in': [], 'output_id': 0, 'shape': [], 'out': [3], 'sorted_id': 2}
{'name': 'Net/Linear[fc2]/weight/weight.9', 'op': 'prim::GetAttr', 'in': [], 'output_id': 0, 'shape': [], 'out': [7], 'sorted_id': 5}
{'name': 'Net/Linear[fc2]/bias/bias.9', 'op': 'prim::GetAttr', 'in': [], 'output_id': 0, 'shape': [], 'out': [7], 'sorted_id': 6}
{'name': 'Net/Linear[fc3]/weight/weight', 'op': 'prim::GetAttr', 'in': [], 'output_id': 0, 'shape': [], 'out': [11], 'sorted_id': 9}
{'name': 'Net/Linear[fc3]/bias/bias', 'op': 'prim::GetAttr', 'in': [], 'output_id': 0, 'shape': [], 'out': [11], 'sorted_id': 10}
[PARAM] ./test/test2_param.cpp
{'name': 'input/x', 'op': 'IO Node', 'in': [], 'output_id': 0, 'shape': [1, 2], 'out': [3], 'sorted_id': 0}
{'name': 'Net/Linear[fc1]/weight/weight.7'

pythonコードを実行する

In [12]:
# check code
#torch.set_grad_enabled(True)  #　効かない(?)
yy = model( input_to_model )
print(yy)
yy.sum().backward()
print(input_to_model.grad)

tensor([[ 0.1536, -0.1925,  0.0064]], grad_fn=<AddmmBackward0>)
tensor([[-0.1284,  0.1121]])


C++コードをコンパイル、リンクする

In [15]:
!g++ -std=c++14 ./test/test2.cpp ./test/test2_param.cpp -D_NOTEBOOK -I ../../../xtensor -lcblas -o ./test/test2

(注意) xtensorにxtensor関連のincludeファイルを置いています。各自の環境に合わせて変更して下さい。

In [16]:
! ./test/test2

### forward computation ...
{{ 0.153593, -0.192527,  0.006398}}
### backward computation ...
input_grad{{-0.128356,  0.112093}}
